# 1. Data processing

## Scrape wikipedia for Toronto neighborhood data

In [479]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [480]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.text, 'html.parser' )
table_html = soup.find_all('table')

body = table_html[0].find_all('tr')


## Convert html to pandas dataframe

In [481]:
#Capture all data, clean and convert from html 

all_rows =[]
for row_num in range(len(body)):
    rows =[]
    
    for row_item in body[row_num].find_all('td'):
        aa = re.sub("(\xa0)|(\n)|","",row_item.text) #Remove newlines and other characters from data
        rows.append(aa)       
    all_rows.append(rows)    

#Flatten nested lists and add to dataframe for further processing
flatten = [item for elem in all_rows for item in elem]
flat_df = pd.DataFrame(flatten)
flat_df

#Clean up and remove 'Not assigned' rows from flattened dataframe
searchfor = ['Not assigned']
clean_df = flat_df[~flat_df[0].str.contains('|'.join(searchfor))]
clean_df.reset_index(drop= True, inplace=True)
clean_df

#Convert dataframe to list for regex selection
df_to_list = clean_df[0].to_list()
df_to_list

post_codes =[]
neighborhood =[]
borough = []

for item in df_to_list:
    pc = re.findall(r'[A-Z]\d[A-Z]', item) # Isolate post codes
    nb = re.findall(r'\([^)]*\)', item)# Isolate neighborhoods
    b = re.sub(r'[A-Z]\d[A-Z]|\([^)]*\)', ' ', item)# Isolate boroughs 
    post_codes.append(pc)
    neighborhood.append(nb)
    borough.append(b)
    
#Flatten post code nested lists
flat_post_codes = [item for elem in post_codes for item in elem]

#Create inital dataframe and clean up some data that wasnt properly processed from the regex
post_code_df = pd.DataFrame(flat_post_codes)
post_code_df.columns = ['Postal Codes']
post_code_df.drop([77, 94, 103], inplace=True)# Remove postcodes at these indexs
post_code_df.reset_index(drop=True)

#Add Borough column and edit some rows
post_code_df.insert(loc=1, column = 'Borough', value = borough) 

post_code_df.loc[35, 'Borough'] = 'East York'
post_code_df.loc[76, 'Borough'] = 'Mississauga'
post_code_df.loc[93, 'Borough'] = 'Downtown Toronto'
post_code_df.loc[96, 'Borough'] = 'Etobicoke'
post_code_df.loc[102, 'Borough'] = 'East Toronto'

#Convert neighborhoods to column for inserting in dataframe, important to match index up correctly
post_code_df.insert(loc=2, column ='Neighborhood', value = neighborhood)
post_code_df['Neighborhood'] = [','.join(map(str, l)) for l in post_code_df['Neighborhood']]


# #Remove brackets from neighborhood column
post_code_df['Neighborhood'] = post_code_df['Neighborhood'].str.strip('()')
post_code_df.loc[13, 'Neighborhood'] = 'Don Mills, Flemingdon Park'
post_code_df['Neighborhood'].replace('/', ',', regex=True, inplace=True)
# post_code_df
post_code_df


# Sort the df by postcode
sort_df = post_code_df.sort_values(['Postal Codes']).reset_index(drop=True)
sort_df.head()


,Postal Codes,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [482]:
#Dataframe size
sort_df.shape

(103, 3)

## Read in geospacial csv file

In [483]:
csv_path= '/Users/alexbaylis/Downloads/Geospatial_Coordinates.csv'
geo_df = pd.read_csv(csv_path)

# Only want latitude and longitude info from df
geo2 = geo_df[['Latitude', 'Longitude']]

In [484]:
#Merge the two dataframes
stack_df = pd.concat([sort_df, geo2], axis=1)
stack_df.head()

,Postal Codes,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# 2. Explore and Cluster Neighborhoods in Toronto

## Import libraries and packages

In [485]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Libraries imported.


In [486]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


In [487]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(stack_df['Latitude'], stack_df['Longitude'], stack_df['Borough'], stack_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [488]:
# Create refined dataframe of central Toronto neighborhoods for clustering later
toronto_data = stack_df[(stack_df['Borough'].isin(['West Toronto', 
                                                   'Central Toronto', 
                                                   'Downtown Toronto', 
                                                   'East Toronto']))].reset_index(drop=True)

toronto_data

,Postal Codes,Borough,Neighborhood,Latitude,Longitude
0,M5W,Downtown Toronto,Enclave of M5E,43.646435,-79.374846
1,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558


## Declare Foursquare Access Tokens

In [489]:
CLIENT_ID = 'OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2' # your Foursquare ID
CLIENT_SECRET = 'SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR' # your Foursquare Secret
ACCESS_TOKEN = '5YLZILZU1QSFO25QG4XYUO350BH33ZXQPJB0PD0VZUUT5MIO'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2
CLIENT_SECRET:SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR


In [490]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, latitude, longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=OCOCDYUJ25V5B5DUJJCBRQROYERUC1ASPSLSQGZAYDYE4L2&client_secret=SQYE2ZURUKDXYCYST21NLL5AMV4DCGXK2M2FNVJFFGKDUCSR&oauth_token=5YLZILZU1QSFO25QG4XYUO350BH33ZXQPJB0PD0VZUUT5MIO&ll=43.6534817,-79.3839347&v=20180605&radius=500&limit=100'

In [491]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60508b55840f4109d7cf85da'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 164,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatL

In [492]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [493]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/Users/alexbaylis/opt/anaconda3/envs/Deadlyrics37/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Indigo,Bookstore,43.653515,-79.380696
4,Poke Guys,Poke Place,43.654895,-79.385052


In [494]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


## Explore Toronto Neighborhoods 


In [495]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            ACCESS_TOKEN,
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [496]:
toronto_venues = getNearbyVenues(names=stack_df['Neighborhood'],
                                  latitudes=stack_df['Latitude'],
                                  longitudes=stack_df['Longitude'],
                                  radius=500)


Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview , Henry Farm , Oriole
Bayview Village
York Mills , Silver Hills
Willowdale , Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills, Flemingdon Park
Bathurst Manor , Wilson Heights , Downsview North
Northwood Park , York University
Downsview),(CFB Toronto
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill , Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West , 

In [497]:
print(toronto_venues.shape)
toronto_venues.head()

(3076, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern , Rouge",43.806686,-79.194353,NT Home Service Inc.,43.806411,-79.197736,Home Service
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
4,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service


In [498]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,7,7,7,7,7,7
"Alderwood , Long Branch",11,11,11,11,11,11
"Bathurst Manor , Wilson Heights , Downsview North",34,34,34,34,34,34
Bayview Village,6,6,6,6,6,6
"Bedford Park , Lawrence Manor East",53,53,53,53,53,53
Berczy Park,100,100,100,100,100,100
"Birch Cliff , Cliffside West",5,5,5,5,5,5
"Brockton , Parkdale Village , Exhibition Place",31,31,31,31,31,31
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",19,19,19,19,19,19


In [499]:
print('There are {} uniques venue categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 317 uniques venue categories.


## Analyze Each Neighborhood

First we process the data using one hot encoding.

In [500]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
neighborhood_col = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neighborhood_col)


toronto_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southe

In [501]:
toronto_onehot.shape

(3076, 317)

Next, we group rows by neighborhood and take the mean of the frequency of occurence of each category.

In [502]:
toronto_grouped = toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
toronto_grouped.head()


,Neighborhood,ATM,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Bath House,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southe

In [503]:
toronto_grouped.shape

(99, 317)

Now we print each neighborhood along with the top 5 most common venues.

In [504]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0            Fireworks Store  0.14
1               Skating Rink  0.14
2             Clothing Store  0.14
3                     Lounge  0.14
4  Latin American Restaurant  0.14


----Alderwood , Long Branch----
            venue  freq
0     Pizza Place  0.18
1        Pharmacy  0.09
2             Pub  0.09
3  Sandwich Place  0.09
4    Skating Rink  0.09


----Bathurst Manor , Wilson Heights , Downsview North----
               venue  freq
0     Ice Cream Shop  0.06
1           Pharmacy  0.06
2  Mobile Phone Shop  0.06
3               Bank  0.06
4                Spa  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.17
1                 Bank  0.17
2                  Spa  0.17
3  Japanese Restaurant  0.17
4                 Café  0.17


----Bedford Park , Lawrence Manor East----
                venue  freq
0                 Spa  0.08
1         Pizza Place  0.06
2  Italian Restaurant  0.06
3    Sushi Rest

4   Coffee Shop  0.11


----The Danforth  East----
                 venue  freq
0    Convenience Store  0.25
1        Metro Station  0.25
2          Film Studio  0.25
3                 Park  0.25
4  Monument / Landmark  0.00


----The Danforth West , Riverdale----
                venue  freq
0    Greek Restaurant  0.14
1         Coffee Shop  0.06
2                 Spa  0.05
3    Sushi Restaurant  0.05
4  Italian Restaurant  0.03


----The Kingsway , Montgomery Road , Old Mill North----
                 venue  freq
0          Yoga Studio  0.25
1                River  0.25
2                 Park  0.25
3           Smoke Shop  0.25
4  Monument / Landmark  0.00


----Thorncliffe Park----
                  venue  freq
0     Mobile Phone Shop  0.11
1              Pharmacy  0.08
2        Sandwich Place  0.05
3  Fast Food Restaurant  0.05
4           Pizza Place  0.05


----Toronto Dominion Centre , Design Exchange----
                venue  freq
0         Coffee Shop  0.13
1                Caf

Lets put this in a pandas dataframe.


In [505]:
#Function to sort venues in descending order:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


#Create dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Hardware Store,Fireworks Store,Skating Rink,Latin American Restaurant,Lounge,Clothing Store,Escape Room,Drugstore,Dumpling Restaurant
1,"Alderwood , Long Branch",Pizza Place,Pharmacy,Pool,Sandwich Place,Athletics & Sports,Pub,Dance Studio,Coffee Shop,Gym,Skating Rink
2,"Bathurst Manor , Wilson Heights , Downsview North",Coffee Shop,Ice Cream Shop,Bank,Mobile Phone Shop,Spa,Pharmacy,Supermarket,Deli / Bodega,Intersection,Frozen Yogurt Shop
3,Bayview Village,Bank,Japanese Restaurant,Spa,Chinese Restaurant,Café,Gym,Yoga Studio,Escape Room,Dumpling Restaurant,Eastern European Restaurant
4,"Bedford Park , Lawrence Manor East",Spa,Italian Restaurant,Pizza Place,Hobby Shop,Massage Studio,Sandwich Place,Sushi Restaurant,Business Service,Coffee Shop,Mobile Phone Shop
5,Berczy Park,Coffee Shop,Sandwich Place,Bakery,Cocktail Bar,Italian Restaurant,Pub,Restaurant,French Restaurant,Beer Bar,Steakhouse
6,"Birch Cliff , Cliffside West",General Entertainment,College Stadium,Farm,Skating Rink,Café,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
7,"Brockton , Parkdale Village , Exhibition Place",Café,Nightclub,Breakfast Spot,Sandwich Place,Bakery,Coffee Shop,Performing Arts Venue,Bar,Tech Startup,Furniture / Home Store
8,"CN Tower , King and Spadina , Railway Lands , ...",Airport Service,Airport Terminal,Rental Car Location,Airport Lounge,Harbor / Marina,Boat or Ferry,Plane,Coffee Shop,Sculpture Garden,Boutique
9,Caledonia-Fairbanks,Park,Miscellaneous Shop,Women's Store,Spa,Yoga Studio,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant


# 3. Cluster Neighborhoods
## K-means clustering 

In [528]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# Its a good idea to scale the features before using k-means clustering
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler_cluster = scaler.fit_transform(toronto_grouped_clustering)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=42).fit(scaler_cluster)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:100])

#Silhouette score is a useful metric to understand how many clusters to use: 
# Scores are between -1 and 1.
# Close to  1 indicating instances are well inside their cluster
# Close to 0 means they are on the cluster boundary
# Close to -1 means they are likely in the wrong cluster!  
from sklearn.metrics import silhouette_score
print(f' \n The silhouette score is: {silhouette_score(scaler_cluster, kmeans.labels_)}')

[1 1 4 4 1 1 1 1 4 1 4 4 4 3 4 4 1 0 4 1 1 1 1 1 4 1 1 4 1 1 1 1 0 1 1 1 1
 1 0 4 4 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 1 1 1 0 1 1 1 4 4 1
 4 1 4 4 1 1 1 1 1 4 1 4 0 4 1 1 4 1 1 4 1 1 1 1 1]
 
 The silhouette score is: 0.039559870547354034


Create a new dataframe that includes the cluster labels as well as the top 10 venues for each neighborhood.

In [529]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = stack_df


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# Remove NaN row
toronto_merged.dropna(axis=0, how='any', inplace=True)

#Convert float values to int for use in folium
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(np.int64)
toronto_merged.head(100)

,Postal Codes,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,1,Home Service,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,1,Construction & Landscaping,Home Service,Moving Target,Golf Course,Yoga Studio,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1,Electronics Store,Rental Car Location,Intersection,Medical Center,Spa,Mexican Restaurant,Bank,Restaurant,Breakfast Spot,Cycle Studio
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1,Coffee Shop,Insurance Office,Other Repair Shop,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Caribbean Restaurant,Bakery,Burger Joint,Hakka Restaurant,Gas Station,Gym,Lounge,Spa,Athletics & Sports,Thai Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,1,Women's Store,Financial or Legal Service,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,1,Pharmacy,Discount Store,Hobby Shop,Train Station,Bus Station,Chinese Restaurant,Department Store,Coffee Shop,Convenience Store,Comfort Food Restaurant
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,1,Bakery,Bus Line,Metro Station,Intersection,Business Service,Ice Cream Shop,Bus Station,Soccer Field,Park,Falafel Restaurant
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,1,Motel,American Restaurant,Movie Theater,Intersection,Skating Rink,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,1,General Entertainment,College Stadium,Farm,Skating Rink,Café,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant


Visualise the resulting clusters:

In [530]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# 4. Examine the clusters

Here we examine each cluster and determine the discriminating venue categories that distinguish each cluster.

Based on the defining categories, we can assign a name to each cluster.

## Cluster 0
### Downtown Toronto district

In [536]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
58,Downtown Toronto,0,Coffee Shop,Café,Asian Restaurant,Salad Place,Hotel,Steakhouse,Sushi Restaurant,Gym,American Restaurant,Thai Restaurant
59,Downtown Toronto,0,Coffee Shop,Aquarium,Café,Hotel,Restaurant,Brewery,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Bar
60,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Seafood Restaurant,Italian Restaurant,Steakhouse,Deli / Bodega,Japanese Restaurant,Lounge
61,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Hotel,Gym,Italian Restaurant,Japanese Restaurant,Seafood Restaurant,Cocktail Bar,Deli / Bodega
70,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Gym,Salad Place,Seafood Restaurant,Deli / Bodega,Steakhouse


## Cluster 1 
### Outskirts of town, parks, buisness parks

In [537]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Home Service,Fast Food Restaurant,Yoga Studio,Ethiopian Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
1,Scarborough,1,Construction & Landscaping,Home Service,Moving Target,Golf Course,Yoga Studio,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
2,Scarborough,1,Electronics Store,Rental Car Location,Intersection,Medical Center,Spa,Mexican Restaurant,Bank,Restaurant,Breakfast Spot,Cycle Studio
3,Scarborough,1,Coffee Shop,Insurance Office,Other Repair Shop,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant
5,Scarborough,1,Women's Store,Financial or Legal Service,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Escape Room,Ethiopian Restaurant
6,Scarborough,1,Pharmacy,Discount Store,Hobby Shop,Train Station,Bus Station,Chinese Restaurant,Department Store,Coffee Shop,Convenience Store,Comfort Food Restaurant
7,Scarborough,1,Bakery,Bus Line,Metro Station,Intersection,Business Service,Ice Cream Shop,Bus Station,Soccer Field,Park,Falafel Restaurant
8,Scarborough,1,Motel,American Restaurant,Movie Theater,Intersection,Skating Rink,Escape Room,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
9,Scarborough,1,General Entertainment,College Stadium,Farm,Skating Rink,Café,Yoga Studio,Empanada Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
10,Scarborough,1,Indian Restaurant,Shop & Service,Thrift / Vintage Store,Light Rail Station,Chinese Restaurant,Mexican Restaurant,Fish & Chips Shop,Pet Store,Vietnamese Restaurant,Construction & Landscaping


## Cluster 2
### Chinatown

In [538]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
67,Downtown Toronto,2,Café,Vietnamese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bar,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,Farmers Market


## Cluster 3 
### Resturant District

In [539]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
52,Downtown Toronto,3,Coffee Shop,Gay Bar,Sushi Restaurant,Japanese Restaurant,Burrito Place,Burger Joint,Yoga Studio,Café,Mexican Restaurant,Men's Store


## Cluster 4
### Outskirts, grocery stores, outlets

In [535]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,4,Caribbean Restaurant,Bakery,Burger Joint,Hakka Restaurant,Gas Station,Gym,Lounge,Spa,Athletics & Sports,Thai Restaurant
13,Scarborough,4,Convenience Store,Fast Food Restaurant,Pharmacy,Rental Car Location,Pizza Place,Gas Station,Spa,Business Service,Flower Shop,Fried Chicken Joint
15,Scarborough,4,Mobile Phone Shop,Fast Food Restaurant,Coffee Shop,Clothing Store,Electronics Store,Furniture / Home Store,Discount Store,Chinese Restaurant,Supermarket,Camera Store
17,North York,4,Golf Course,Mediterranean Restaurant,Fast Food Restaurant,Athletics & Sports,Pool,Dog Run,Yoga Studio,Donut Shop,Drugstore,Dumpling Restaurant
19,North York,4,Bank,Japanese Restaurant,Spa,Chinese Restaurant,Café,Gym,Yoga Studio,Escape Room,Dumpling Restaurant,Eastern European Restaurant
22,North York South,4,Rental Car Location,Restaurant,Coffee Shop,Ramen Restaurant,Pizza Place,Japanese Restaurant,Café,Bubble Tea Shop,Sandwich Place,Discount Store
24,North York West,4,Rental Car Location,Restaurant,Coffee Shop,Ramen Restaurant,Pizza Place,Japanese Restaurant,Café,Bubble Tea Shop,Sandwich Place,Discount Store
28,North York,4,Coffee Shop,Ice Cream Shop,Bank,Mobile Phone Shop,Spa,Pharmacy,Supermarket,Deli / Bodega,Intersection,Frozen Yogurt Shop
31,North York West,4,Grocery Store,Shopping Mall,Home Service,Mobile Phone Shop,Pharmacy,Korean Restaurant,Athletics & Sports,Discount Store,Baseball Field,Liquor Store
32,North York Central,4,Grocery Store,Shopping Mall,Home Service,Mobile Phone Shop,Pharmacy,Korean Restaurant,Athletics & Sports,Discount Store,Baseball Field,Liquor Store
